In [194]:
# -*- coding: utf-8 -*-
# import different relevant libraries

import numpy as np
import pandas as pd

# pd.set_option('max_columns',200)

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import difflib
from collections import defaultdict
import jellyfish
import re
import fuzzy
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

height has been deprecated.



In [195]:
# read csv file

data = pd.read_csv('Deduplication Problem - Sample Dataset.csv')

In [196]:
data.head()

,ln,dob,gn,fn
0,SMITH JR,01/03/68,F,WILLIAM
1,ROTHMEYER JR,01/03/68,F,WILLIAM
2,ASBY JR,01/03/68,F,WILLIAM
3,SALTER JR,01/03/68,F,WILLIAM
4,SALTER JR,01/03/68,F,WILLIAM


In [197]:
data.describe()

,ln,dob,gn,fn
count,103,103,103,103
unique,28,22,2,26
top,MICHAELSON JR,07/10/37,M,HAROLD
freq,22,24,77,24


In [198]:
df = pd.DataFrame()

In [199]:
data = data[['ln','fn']]

In [200]:
data

,ln,fn
0,SMITH JR,WILLIAM
1,ROTHMEYER JR,WILLIAM
2,ASBY JR,WILLIAM
3,SALTER JR,WILLIAM
4,SALTER JR,WILLIAM
5,BLAND JR,WILLIAM
6,BLAND JR,WILLIAM
7,BLAND JR,WILLIAM
8,SHAFFER JR,WILLIAM
9,SHAFFER JR,WILLIAM


In [201]:
df['Full_Name'] = data.ln + ' ' + data.fn

In [202]:
# this function is used to clean string 

def string_clean(input_str, normalized=False, ignore_list=[]):

    for ignore_str in ignore_list:
        input_str = re.sub(r'{0}'.format(ignore_str), '', input_str,
                           flags=re.IGNORECASE)

    if normalized is True:
        input_str = input_str.strip().lower()

        # clean special chars and extra whitespace

        input_str = re.sub("\W", '', input_str).strip()

    return input_str


# this function is used to find the similarity between two names

def find_similarity_score(
    first_str,
    second_str,
    normalized=False,
    ignore_list=[],
    ):

    first_str = string_clean(first_str, normalized=normalized,
                             ignore_list=ignore_list)
    second_str = string_clean(second_str, normalized=normalized,
                              ignore_list=ignore_list)
    match_ratio = (difflib.SequenceMatcher(None, first_str,
                   second_str).ratio()
                   + jellyfish.jaro_winkler(unicode(first_str),
                   unicode(second_str))) / 2.0
    return match_ratio


In [203]:
find_similarity_score('JAMES MICHAELSON JR', 'ROBERT MICHAELSON JR',
                      normalized=True, ignore_list=[])


0.7730960760372525

In [204]:
find_similarity_score('JAMES MICHAELSON JR', 'GALETICH JR ADDISON',
                      normalized=True, ignore_list=[])

0.6016042780748663

In [205]:
matrix = pd.DataFrame(columns=df.Full_Name)
matrix

Full_Name,SMITH JR WILLIAM,ROTHMEYER JR WILLIAM,ASBY JR WILLIAM,SALTER JR WILLIAM,SALTER JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,SHAFFER JR WILLIAM,BLAND III WILLIAM,BLAND JR BILL,BLAND WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,DUNCAN JR THOMAS,CARLSON JR ROY,CARLSON JR RON,DUNCAN JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROBERT,MICHAELSON JR BOB,MUSTAPHA JR OWEN,MUSTAPHA JR OWAN,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,LIND JR LAWRENCE,LIND JR LAWRENCE,LIND JR KENNETH,LIND JR KENNETH,LIND JR JOHN,CAIN JR JOHN,CLARK JOHN,CLARK RONALD,CLARK DONALD,CLARK JOHN,CLARK JR JOHN,CLARK JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JAMES,MICHAELSON JR JAMES,MICHAELSON JR JACK,MICHAELSON JR JACK,LARSON JR HAROLD,LARSON JR HAROLD,MUSTAPHA JR HAROLD,HOUGHTON JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,MICHAELSON JR HAROLD,MICHAELSON JR HAROLD,SMITH JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,FAGEN JR HAROLD,FAGEN JR HAROLD,FAGEN JR GEORGE,GRIFFIN JR DARH,GRIFFIN JR DARL,GRIFFIN JR DARL,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLARENCE,HANNA ADDISON J,HANNA ADDISON JOHN,HANNA ADDISON,HANNA ADDISON,HANNA ADDISON,GALETICH JR ADDISON,FUNARO JONES HARRIET,FUNARO J HARRIET,FUNARO HARRIET


In [206]:
matrix['Full_Name'] = df.Full_Name
matrix

Full_Name,SMITH JR WILLIAM,ROTHMEYER JR WILLIAM,ASBY JR WILLIAM,SALTER JR WILLIAM,SALTER JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,SHAFFER JR WILLIAM,BLAND III WILLIAM,BLAND JR BILL,BLAND WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,DUNCAN JR THOMAS,CARLSON JR ROY,CARLSON JR RON,DUNCAN JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROBERT,MICHAELSON JR BOB,MUSTAPHA JR OWEN,MUSTAPHA JR OWAN,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,LIND JR LAWRENCE,LIND JR LAWRENCE,LIND JR KENNETH,LIND JR KENNETH,LIND JR JOHN,CAIN JR JOHN,CLARK JOHN,CLARK RONALD,CLARK DONALD,CLARK JOHN,CLARK JR JOHN,CLARK JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JAMES,MICHAELSON JR JAMES,MICHAELSON JR JACK,MICHAELSON JR JACK,LARSON JR HAROLD,LARSON JR HAROLD,MUSTAPHA JR HAROLD,HOUGHTON JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,MICHAELSON JR HAROLD,MICHAELSON JR HAROLD,SMITH JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,FAGEN JR HAROLD,FAGEN JR HAROLD,FAGEN JR GEORGE,GRIFFIN JR DARH,GRIFFIN JR DARL,GRIFFIN JR DARL,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLARENCE,HANNA ADDISON J,HANNA ADDISON JOHN,HANNA ADDISON,HANNA ADDISON,HANNA ADDISON,GALETICH JR ADDISON,FUNARO JONES HARRIET,FUNARO J HARRIET,FUNARO HARRIET,Full_Name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SMITH JR WILLIAM
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROTHMEYER JR WILLIAM
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASBY JR WILLIAM
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SALTER JR WILLIAM
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SALTER JR WILLIAM
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [207]:
matrix.set_index('Full_Name', drop=True, inplace=True)

In [208]:
matrix

Full_Name,SMITH JR WILLIAM,ROTHMEYER JR WILLIAM,ASBY JR WILLIAM,SALTER JR WILLIAM,SALTER JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,SHAFFER JR WILLIAM,BLAND III WILLIAM,BLAND JR BILL,BLAND WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,DUNCAN JR THOMAS,CARLSON JR ROY,CARLSON JR RON,DUNCAN JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROBERT,MICHAELSON JR BOB,MUSTAPHA JR OWEN,MUSTAPHA JR OWAN,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,LIND JR LAWRENCE,LIND JR LAWRENCE,LIND JR KENNETH,LIND JR KENNETH,LIND JR JOHN,CAIN JR JOHN,CLARK JOHN,CLARK RONALD,CLARK DONALD,CLARK JOHN,CLARK JR JOHN,CLARK JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JAMES,MICHAELSON JR JAMES,MICHAELSON JR JACK,MICHAELSON JR JACK,LARSON JR HAROLD,LARSON JR HAROLD,MUSTAPHA JR HAROLD,HOUGHTON JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,MICHAELSON JR HAROLD,MICHAELSON JR HAROLD,SMITH JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,FAGEN JR HAROLD,FAGEN JR HAROLD,FAGEN JR GEORGE,GRIFFIN JR DARH,GRIFFIN JR DARL,GRIFFIN JR DARL,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLARENCE,HANNA ADDISON J,HANNA ADDISON JOHN,HANNA ADDISON,HANNA ADDISON,HANNA ADDISON,GALETICH JR ADDISON,FUNARO JONES HARRIET,FUNARO J HARRIET,FUNARO HARRIET
Full_Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SMITH JR WILLIAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ROTHMEYER JR WILLIAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ASBY JR WILLIAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SALTER JR WILLIAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SALTER JR WILLIAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [209]:
# make a similarity matrix with similarity score

for i in matrix.index:
    for j in matrix.columns:
        matrix.loc[i, j] = find_similarity_score(i, j, ignore_list=['JR'
                ])

In [210]:
matrix

Full_Name,SMITH JR WILLIAM,ROTHMEYER JR WILLIAM,ASBY JR WILLIAM,SALTER JR WILLIAM,SALTER JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,SHAFFER JR WILLIAM,BLAND III WILLIAM,BLAND JR BILL,BLAND WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,DUNCAN JR THOMAS,CARLSON JR ROY,CARLSON JR RON,DUNCAN JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROBERT,MICHAELSON JR BOB,MUSTAPHA JR OWEN,MUSTAPHA JR OWAN,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,LIND JR LAWRENCE,LIND JR LAWRENCE,LIND JR KENNETH,LIND JR KENNETH,LIND JR JOHN,CAIN JR JOHN,CLARK JOHN,CLARK RONALD,CLARK DONALD,CLARK JOHN,CLARK JR JOHN,CLARK JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JAMES,MICHAELSON JR JAMES,MICHAELSON JR JACK,MICHAELSON JR JACK,LARSON JR HAROLD,LARSON JR HAROLD,MUSTAPHA JR HAROLD,HOUGHTON JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,MICHAELSON JR HAROLD,MICHAELSON JR HAROLD,SMITH JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,FAGEN JR HAROLD,FAGEN JR HAROLD,FAGEN JR GEORGE,GRIFFIN JR DARH,GRIFFIN JR DARL,GRIFFIN JR DARL,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLARENCE,HANNA ADDISON J,HANNA ADDISON JOHN,HANNA ADDISON,HANNA ADDISON,HANNA ADDISON,GALETICH JR ADDISON,FUNARO JONES HARRIET,FUNARO J HARRIET,FUNARO HARRIET
Full_Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SMITH JR WILLIAM,1,0.750496,0.750956,0.807167,0.807167,0.665344,0.665344,0.665344,0.787649,0.787649,0.624076,0.468615,0.619098,0.665344,0.665344,0.665344,0.665344,0.665344,0.787649,0.333333,0.343559,0.343559,0.300779,0.395238,0.449398,0.395238,0.449398,0.449398,0.449398,0.449398,0.449398,0.408896,0.449398,0.529762,0.565476,0.392196,0.392196,0.392196,0.424603,0.424603,0.351954,0.351954,0.364286,0.364286,0.182143,0.265415,0.265415,0.182143,0.27228,0.27228,0.418155,0.418155,0.418155,0.418155,0.418155,0.418155,0.418155,0.418155,0.532296,0.532296,0.482738,0.482738,0.396825,0.396825,0.503571,0.378373,0.464286,0.464286,0.464286,0.464286,0.464286,0.464286,0.464286,0.464286,0.464286,0.396825,0.396825,0.396825,0.396825,0.470569,0.470569,0.718702,0.396825,0.396825,0.334737,0.334737,0.290191,0.405108,0.407611,0.407611,0.430431,0.430431,0.430431,0.430431,0.286029,0.26832,0.260938,0.260938,0.260938,0.439126,0.322759,0.278274,0.25
ROTHMEYER JR WILLIAM,0.750496,1,0.643378,0.760859,0.760859,0.638393,0.638393,0.638393,0.74183,0.74183,0.558341,0.461134,0.538869,0.638393,0.638393,0.638393,0.638393,0.638393,0.74183,0.363095,0.35,0.315741,0.325502,0.367229,0.385354,0.367229,0.385354,0.385354,0.385354,0.385354,0.385354,0.388889,0.352549,0.399636,0.478505,0.424074,0.424074,0.424074,0.337136,0.337136,0.236169,0.236169,0.258466,0.258466,0.335847,0.35,0.315741,0.335847,0.325502,0.325502,0.345014,0.345014,0.345014,0.345014,0.345014,0.345014,0.345014,0.345014,0.393013,0.393013,0.400054,0.400054,0.367229,0.367229,0.349809,0.349809,0.376819,0.376819,0.376819,0.376819,0.376819,0.376819,0.376819,0.376819,0.376819,0.331845,0.331845,0.331845,0.331845,0.385185,0.385185,0.377579,0.331845,0.331845,0.346571,0.346571,0.312586,0.4004,0.402638,0.402638,0.395347,0.395347,0.395347,0.399529,0.267088,0.25463,0.265164,0.265164,0.265164,0.399529,0.344688,0.37643,0.388062
ASBY JR WILLIAM,0.750956,0.643378,1,0.741603,0.741603,0.793864,0.793864,0.793864,0.751952,0.751952,0.703414,0.588661,0.752564,0.793864,0.793864,0.793864,0.793864,0.793864,0.751952,0.331034,0.42688,0.42688,0.375583,0.334737,0.36978,0.334737,0.36978,

In [211]:
# set threshold

t_matrix = matrix.iloc[:, :] > 0.85

In [212]:
t_matrix

Full_Name,SMITH JR WILLIAM,ROTHMEYER JR WILLIAM,ASBY JR WILLIAM,SALTER JR WILLIAM,SALTER JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,SHAFFER JR WILLIAM,BLAND III WILLIAM,BLAND JR BILL,BLAND WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,DUNCAN JR THOMAS,CARLSON JR ROY,CARLSON JR RON,DUNCAN JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROBERT,MICHAELSON JR BOB,MUSTAPHA JR OWEN,MUSTAPHA JR OWAN,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,LIND JR LAWRENCE,LIND JR LAWRENCE,LIND JR KENNETH,LIND JR KENNETH,LIND JR JOHN,CAIN JR JOHN,CLARK JOHN,CLARK RONALD,CLARK DONALD,CLARK JOHN,CLARK JR JOHN,CLARK JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JAMES,MICHAELSON JR JAMES,MICHAELSON JR JACK,MICHAELSON JR JACK,LARSON JR HAROLD,LARSON JR HAROLD,MUSTAPHA JR HAROLD,HOUGHTON JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,MICHAELSON JR HAROLD,MICHAELSON JR HAROLD,SMITH JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,FAGEN JR HAROLD,FAGEN JR HAROLD,FAGEN JR GEORGE,GRIFFIN JR DARH,GRIFFIN JR DARL,GRIFFIN JR DARL,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLARENCE,HANNA ADDISON J,HANNA ADDISON JOHN,HANNA ADDISON,HANNA ADDISON,HANNA ADDISON,GALETICH JR ADDISON,FUNARO JONES HARRIET,FUNARO J HARRIET,FUNARO HARRIET
Full_Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SMITH JR WILLIAM,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
ROTHMEYER JR WILLIAM,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
ASBY JR WILLIAM,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
SALTER JR WILLIAM,False,False,False,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,

In [213]:
t_matrix.describe()

Full_Name,SMITH JR WILLIAM,ROTHMEYER JR WILLIAM,ASBY JR WILLIAM,SALTER JR WILLIAM,SALTER JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,SHAFFER JR WILLIAM,BLAND III WILLIAM,BLAND JR BILL,BLAND WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,DUNCAN JR THOMAS,CARLSON JR ROY,CARLSON JR RON,DUNCAN JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROBERT,MICHAELSON JR BOB,MUSTAPHA JR OWEN,MUSTAPHA JR OWAN,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,LIND JR LAWRENCE,LIND JR LAWRENCE,LIND JR KENNETH,LIND JR KENNETH,LIND JR JOHN,CAIN JR JOHN,CLARK JOHN,CLARK RONALD,CLARK DONALD,CLARK JOHN,CLARK JR JOHN,CLARK JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JAMES,MICHAELSON JR JAMES,MICHAELSON JR JACK,MICHAELSON JR JACK,LARSON JR HAROLD,LARSON JR HAROLD,MUSTAPHA JR HAROLD,HOUGHTON JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,MICHAELSON JR HAROLD,MICHAELSON JR HAROLD,SMITH JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,FAGEN JR HAROLD,FAGEN JR HAROLD,FAGEN JR GEORGE,GRIFFIN JR DARH,GRIFFIN JR DARL,GRIFFIN JR DARL,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLARENCE,HANNA ADDISON J,HANNA ADDISON JOHN,HANNA ADDISON,HANNA ADDISON,HANNA ADDISON,GALETICH JR ADDISON,FUNARO JONES HARRIET,FUNARO J HARRIET,FUNARO HARRIET
count,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,102,102,102,98,98,92,92,92,98,98,93,94,93,92,92,92,92,92,98,102,101,101,102,83,83,83,83,83,83,83,83,93,83,101,101,100,100,100,101,101,101,101,102,102,99,101,101,99,99,99,82,82,82,82,82,82,82,82,91,91,91,91,101,101,102,102,94,94,94,94,94,94,94,94,94,97,97,97,97,92,92,102,97,97,101,101,102,100,100,100,100,100,100,102,98,98,98,98,98,102,100,100,100


In [214]:
# remove absolute same strings

rt_matrix = t_matrix[~t_matrix.index.duplicated(keep='last')]

In [215]:
rt_matrix.head()

Full_Name,SMITH JR WILLIAM,ROTHMEYER JR WILLIAM,ASBY JR WILLIAM,SALTER JR WILLIAM,SALTER JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,SHAFFER JR WILLIAM,BLAND III WILLIAM,BLAND JR BILL,BLAND WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,DUNCAN JR THOMAS,CARLSON JR ROY,CARLSON JR RON,DUNCAN JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROBERT,MICHAELSON JR BOB,MUSTAPHA JR OWEN,MUSTAPHA JR OWAN,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,LIND JR LAWRENCE,LIND JR LAWRENCE,LIND JR KENNETH,LIND JR KENNETH,LIND JR JOHN,CAIN JR JOHN,CLARK JOHN,CLARK RONALD,CLARK DONALD,CLARK JOHN,CLARK JR JOHN,CLARK JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JAMES,MICHAELSON JR JAMES,MICHAELSON JR JACK,MICHAELSON JR JACK,LARSON JR HAROLD,LARSON JR HAROLD,MUSTAPHA JR HAROLD,HOUGHTON JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,MICHAELSON JR HAROLD,MICHAELSON JR HAROLD,SMITH JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,FAGEN JR HAROLD,FAGEN JR HAROLD,FAGEN JR GEORGE,GRIFFIN JR DARH,GRIFFIN JR DARL,GRIFFIN JR DARL,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLARENCE,HANNA ADDISON J,HANNA ADDISON JOHN,HANNA ADDISON,HANNA ADDISON,HANNA ADDISON,GALETICH JR ADDISON,FUNARO JONES HARRIET,FUNARO J HARRIET,FUNARO HARRIET
Full_Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SMITH JR WILLIAM,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
ROTHMEYER JR WILLIAM,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
ASBY JR WILLIAM,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
SALTER JR WILLIAM,False,False,False,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,

In [216]:
rt_matrix.describe()

Full_Name,SMITH JR WILLIAM,ROTHMEYER JR WILLIAM,ASBY JR WILLIAM,SALTER JR WILLIAM,SALTER JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,SHAFFER JR WILLIAM,BLAND III WILLIAM,BLAND JR BILL,BLAND WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,BLAND JR WILLIAM,SHAFFER JR WILLIAM,DUNCAN JR THOMAS,CARLSON JR ROY,CARLSON JR RON,DUNCAN JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROY,MICHAELSON JR ROBERT,MICHAELSON JR BOB,MUSTAPHA JR OWEN,MUSTAPHA JR OWAN,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,HOUGHTON JR LAWRENCE,LIND JR LAWRENCE,LIND JR LAWRENCE,LIND JR KENNETH,LIND JR KENNETH,LIND JR JOHN,CAIN JR JOHN,CLARK JOHN,CLARK RONALD,CLARK DONALD,CLARK JOHN,CLARK JR JOHN,CLARK JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JOHN,MICHAELSON JR JAMES,MICHAELSON JR JAMES,MICHAELSON JR JACK,MICHAELSON JR JACK,LARSON JR HAROLD,LARSON JR HAROLD,MUSTAPHA JR HAROLD,HOUGHTON JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,MELVIN JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,MICHAELSON JR HAROLD,MICHAELSON JR HAROLD,SMITH JR HAROLD,CHAVAS JR HAROLD,CHAVAS JR HAROLD,FAGEN JR HAROLD,FAGEN JR HAROLD,FAGEN JR GEORGE,GRIFFIN JR DARH,GRIFFIN JR DARL,GRIFFIN JR DARL,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLIFFORD,GRIFFIN JR CLARENCE,HANNA ADDISON J,HANNA ADDISON JOHN,HANNA ADDISON,HANNA ADDISON,HANNA ADDISON,GALETICH JR ADDISON,FUNARO JONES HARRIET,FUNARO J HARRIET,FUNARO HARRIET
count,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,50,50,50,49,49,47,47,47,49,49,48,49,48,47,47,47,47,47,49,50,49,49,50,45,45,45,45,45,45,45,45,47,45,49,49,50,50,50,50,50,50,50,50,50,49,49,49,49,49,49,45,45,45,45,45,45,45,45,48,48,48,48,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,47,47,50,50,50,50,50,50,49,49,49,50,50,50,50,48,48,48,48,48,50,48,48,48


In [217]:
# find similar names for each patient name

dic = defaultdict(list)
for i in rt_matrix.index:
    for j in rt_matrix.columns:
        if rt_matrix.loc[i, j].all():
            if j != i and j not in dic[i]:
                dic[i].append(j)

In [218]:
p = []
for i in dic.keys():
    p.append([i, dic[i]])

In [219]:
f = pd.DataFrame(p)

In [220]:
f = f.rename(columns={0: 'Name', 1: 'Similar Names'})

In [221]:
f = f.set_index('Name')

In [222]:
f['Similar Names'] = f['Similar Names'].apply(lambda x: ' , '.join(x))

In [223]:
# Ambiguos patients names with their similars names

f

,Similar Names
Name,
CARLSON JR RON,CARLSON JR ROY
FUNARO HARRIET,"FUNARO JONES HARRIET , FUNARO J HARRIET"
FUNARO JONES HARRIET,"FUNARO J HARRIET , FUNARO HARRIET"
BLAND JR BILL,BLAND JR WILLIAM
CLARK JR JOHN,CLARK JOHN
HANNA ADDISON JOHN,"HANNA ADDISON J , HANNA ADDISON"
CLARK RONALD,CLARK DONALD
MICHAELSON JR JOHN,"MICHAELSON ROY , MICHAELSON JR ROY , MICHAELSO..."
CARLSON JR ROY,CARLSON JR RON


In [224]:
f.to_csv('submission.csv', index=True)